# Covid-19 Averse Vaccine Response Prediction
![vaccine brands](../data/images/vaccines.jpg)
Author: Christos Maglaras<br>
Date : 4/14/2021
## Stakeholder
This project is mainly focused on serving the induvidual by providing an prediction based on personal information of the possibility of a negative reaction from one of the Covid-19 vaccines. It could also be applied in a medical center such as a clinic or hospital to screen patients easily and quickly. If you would like to know your or anothers risk of illness from one of the three available vaccines you may enter some or all of your personal information into the following web form ~flask link~. Successfully being able to predict the outcome of a patient utilizing nothing more than an online form would be higly beneficial to the patient of course, but also the healthcare system by decreasing the amount of strain placed on hospitals. 

## Data
The data utilized for this project has been sourced from the CDC VAERS system, a public dataset consisting of thirty years of domestic adverse vaccine events. Medical professionals and vaccine manufactures are required to report all adverse reactions that come to their attention. While they are required to submit records, anyone can submit a report of their experience. The data consists of general informations such as age and sex, vaccination information like the administration facility and brand, and health information such as preexisting illnesses, allergies, and medications they may take. This dataset contains roughly 70,000 records containing covid-19 vaccines, and is updated every two weeks with new records. You can collect the data [Here](https://vaers.hhs.gov/data/datasets.html?).
![vaers](../data/images/vaers.png)

## Business Understanding
This system would alleviate some of the pressure from hospitals, freeing up resources so they can operate more effectively. The first way in which a system like this would help is as a first step screening method, filtering patients to at least notify their clinician of their risks. The second is that in avoiding the adverse reactions, the hospitals do not need to dedicate extra resources to the patient after the reaction. Aulthough we have here seventy thousand cases reporting adverse reactions, the US now has reached five million vaccinations of at least one dose, and three million full vaccinations, meaning that these adverse reactions are only 1.4% of all domestic vaccinations. This is not to say that the 1.4% are to be ignored, with the legal age of the vaccine being sixteen, that leaves two hundred thirty million people eligible for the vaccine in the us, 1.4% being three million.

## Model
The model that achieved the best results was xgboost optimized using the Bayesian Optimization technique. This achieved significantly better results than both the standard random forests and the neural network which were not able to produce informative models. Aulthough there is some difficulty differentiating between hospital-bound and those who are not, that is between patients who all had some sort of averse reaction. A real wold test might be more precice as it has been trained on the more difficult task of seperating two groups who are very similar to one another. 

## Contents
```
├── data
|   ├── vaers_guide.pdf
|   ├── images
|   |   ├── vaccines.jpg
|   |   └── vaers.mp4
|   └── dataset
|       ├── 2020
|       |   ├── data20.csv
|       |   ├── symptoms20.csv
|       |   └── vax20.csv
|       └── 2021
|           ├── data21.csv
|           ├── symptoms21.csv
|           └── vax21.csv
├── notebooks
|   ├── Exploration Notebook.ipynb
|   ├── Final Notebook.ipynb
|   └── MVP Final Notebook.ipynb
├── presentation.pdf
└── README.md
```

## Data Preprocessing

In [32]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report,mean_squared_error, roc_auc_score, make_scorer
from sklearn.metrics import recall_score

import lazypredict
from lazypredict.Supervised import LazyClassifier

import xgboost as xgb
from bayes_opt import BayesianOptimization

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout

pd.set_option('display.max_columns', None)
import warnings; warnings.simplefilter('ignore')

ImportError: cannot import name 'metrics' from 'sklearn' (unknown location)

In [41]:
pd.set_option('display.max_columns', None)
import warnings; warnings.simplefilter('ignore')

In [33]:
# lets load in both datasets for each year
symptoms20 = pd.read_csv('../data/2020/symptoms20.csv', index_col=['VAERS_ID'], encoding='latin-1')
data20     = pd.read_csv('../data/2020/data20.csv', index_col=['VAERS_ID'], encoding='latin-1')
vax20      = pd.read_csv('../data/2020/vax20.csv', index_col=['VAERS_ID'], encoding='latin-1')

symptoms21 = pd.read_csv('../data/2021/symptoms21.csv', index_col=['VAERS_ID'], encoding='latin-1')
data21     = pd.read_csv('../data/2021/data21.csv', index_col=['VAERS_ID'], encoding='latin-1')
vax21      = pd.read_csv('../data/2021/vax21.csv', index_col=['VAERS_ID'], encoding='latin-1')

C:\Users\Chris\anaconda3\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (7,12,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [34]:
combined_vax = pd.concat([vax20, vax21])
combined_data = pd.concat([data20, data21])
combined_symptoms = pd.concat([symptoms20, symptoms21])

In [35]:
datavax = pd.merge(combined_data, combined_vax, on='VAERS_ID', how='right')
dvs = pd.merge(datavax, combined_symptoms, on='VAERS_ID', how='left')

In [36]:
# isolating covid-19 vaccinations for the base dataframe
df = dvs[dvs['VAX_TYPE'] == 'COVID19']

In [37]:
df.drop_duplicates(inplace=True)

<ipython-input-37-16cdb8520be8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [38]:
df['DIED'] = df['DIED'].fillna(0)
df['DIED'] = df['DIED'].replace('Y', 1)

df['SEX'] = df['SEX'].replace('U', '0')
df['SEX'] = df['SEX'].replace('F', '0')
df['SEX'] = df['SEX'].replace('M', '1')

df['L_THREAT'] = df['L_THREAT'].fillna(0)
df['L_THREAT'] = df['L_THREAT'].replace('Y', 1)

df['HOSPITAL'] = df['HOSPITAL'].fillna(0)
df['HOSPITAL'] = df['HOSPITAL'].replace('Y', 1)

df['HOSPDAYS'] = df['HOSPDAYS'].fillna(0)

df['X_STAY'] = df['X_STAY'].fillna(0)
df['X_STAY'] = df['X_STAY'].replace('Y', 1)

df['DISABLE'] = df['DISABLE'].fillna(0)
df['DISABLE'] = df['DISABLE'].replace('Y', 1)

df['RECOVD'] = df['RECOVD'].fillna(0)
df['RECOVD'] = df['RECOVD'].replace('U', 0)
df['RECOVD'] = df['RECOVD'].replace('N', 0)
df['RECOVD'] = df['RECOVD'].replace('Y', 1)

df['BIRTH_DEFECT'] = df['BIRTH_DEFECT'].fillna(0)
df['BIRTH_DEFECT'] = df['BIRTH_DEFECT'].replace('Y', 1)

df['VAX_DOSE_SERIES'] = df['VAX_DOSE_SERIES'].fillna(0)
df['VAX_DOSE_SERIES'] = df['VAX_DOSE_SERIES'].replace('7+', 7)
df['VAX_DOSE_SERIES'] = df['VAX_DOSE_SERIES'].replace('UNK', 1)

df['NUMDAYS'] = df['NUMDAYS'].where(df['NUMDAYS']<120, 7)

<ipython-input-38-6f50c7f5c66d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DIED'] = df['DIED'].fillna(0)
<ipython-input-38-6f50c7f5c66d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DIED'] = df['DIED'].replace('Y', 1)
<ipython-input-38-6f50c7f5c66d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

In [39]:
df['SEX'] = df['SEX'].astype(int)
df['AGE_YRS'] = df['AGE_YRS'].fillna(50)
df['AGE_YRS'] = df['AGE_YRS'].astype(int)
df['HOSPDAYS'] = df['HOSPDAYS'].astype(int)
df['NUMDAYS'] = df['NUMDAYS'].astype(int)
df['VAX_DOSE_SERIES'] = df['VAX_DOSE_SERIES'].astype(int)

<ipython-input-39-14e06d3e3430>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SEX'] = df['SEX'].astype(int)
<ipython-input-39-14e06d3e3430>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['AGE_YRS'] = df['AGE_YRS'].fillna(50)
<ipython-input-39-14e06d3e3430>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

In [51]:
df['VAX_SITE'].value_counts()

LA    39272
RA    15127
AR     1380
UN     1207
OT       35
LL       29
RL       10
LG        3
GM        3
Name: VAX_SITE, dtype: int64

## Data Processing
### State

In [45]:
# lets clean up the state column by filling NaN values and binning the unusual locations as Other as there are only a few of each
df['STATE'].replace(['AS', 'VI', 'MP', 'Ca', 'XB', 'FM', 'MH', 'GU'], 'OTH', inplace=True)
df['STATE'] = df['STATE'].fillna('NA')

label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(df['STATE'])
df['STATE'] = label_encoder.transform(df['STATE'])

NameError: name 'LabelEncoder' is not defined

### Vaccine Manufacturer

In [ ]:
# since there are no NaNs or unwanted values we can just fit and transform
df['VAX_MANU'].value_counts()

label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(df['VAX_MANU'])
df['VAX_MANU'] = label_encoder.transform(df['VAX_MANU'])

### Injection Site

In [ ]:
# where the dose was applied, right arm, left arm, leg, ect.
df['VAX_SITE'] = df['VAX_SITE'].fillna('NA')

label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(df['VAX_SITE'])
df['VAX_SITE'] = label_encoder.transform(df['VAX_SITE'])

### Delivery Method

In [ ]:
# states the method of delivering the vaccine, such as srynge, nasal, intradermal, intramuscular, and others
df['VAX_ROUTE'] = df['VAX_ROUTE'].fillna('UN')

label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(df['VAX_ROUTE'])
df['VAX_ROUTE'] = label_encoder.transform(df['VAX_ROUTE'])

### Administration Facility

In [ ]:
# examples of locations would be school, military, senior home ect.
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(df['V_ADMINBY'])
df['V_ADMINBY'] = label_encoder.transform(df['V_ADMINBY'])

### Allergies

In [ ]:
# the following columns cleaned cannot using LabelEncoder as many rows have multiple values to represent.

In [ ]:
# preparing the columns with text data is a little tricky as there is no standard format, many misspellings, and multiple values presented for many people
# Lets begin by lowercasing the Allergies column
df['ALLERGIES'] = df['ALLERGIES'].str.lower()

In [ ]:
# Now we replace all these different ways of saying none with none, as well as NaNs
nonelist = ['no', 'no known allergies', 'unknown', 'none known', 'n/a', 'none reported', 'na', 'none.',
            'no known drug allergies', 'no allergies', 'na', 'no known', 'no known allergies.', 'none listed', 
           'unk', 'none known.']

df['ALLERGIES'] = df['ALLERGIES'].fillna('none')
df['ALLERGIES'] = df['ALLERGIES'].replace('penicillin|sulfa', 'penicillin')
df['ALLERGIES'] = df['ALLERGIES'].replace(nonelist, 'none')

In [ ]:
# these functions will find each list-like text and turn it into a format acceptable for get_dummies
# We are using this method instead of onehotencoder or labelbinarizer so a row can have multiple allergies represented
# by only using the allergies that at least thirty people have in the data, we can keep the dimensionality and required cleaning down
# it is likely that an algorith would not be able to learn from allergies that have a very small representation
allall = []
for each in df['ALLERGIES']:
    if ',' in each:
        alls = each.split(',')
        ally = []
        for weach in alls:
            ally.append(weach.strip())
        allall.append(ally)
    else:
        allall.append(each)
        
allall2 = []
for each in allall:
    if type(each) == list:
        welt = "|".join(each)
        allall2.append(welt)
    else:
        allall2.append(each)
        
listy = list(pd.Series(allall2).value_counts()[pd.Series(allall2).value_counts()>30].index)
allall3 = list(map(lambda x: 'none' if x not in listy else x, allall2))

dummyframe = pd.Series(allall3).str.get_dummies()

listy.remove('penicillin|sulfa')

In [ ]:
df.reset_index(inplace=True)

In [ ]:
# Lets attach our new columns to the dataframe
df = df.join(dummyframe[listy])

### Current Illnesses

In [ ]:
df['CUR_ILL'] = df['CUR_ILL'].str.lower()

nonelist = ['no', 'unknown', 'none.', 'none reported', 'n/a', 'na', 'none known', 'denies', 'none noted', '0', 'no illness',
           'none listed', 'not known', 'no known', 'non', 'no acute illnesses', 'no.', 'denied', 'see below', 'no illnesses',
            'unk', 'unkown', 'none documented', 'none stated', 'nothing', 'none known.', 'unknown.', 'no known illnesses',
            'n/a.','no e', 'none reported.', 'no acute illness']

df['CUR_ILL'] = df['CUR_ILL'].fillna('none')
df['CUR_ILL'] = df['CUR_ILL'].replace(nonelist, 'none')
df['CUR_ILL'] = df['CUR_ILL'].replace(['covid 19', 'covid', 'covid- 19 diagnosis 12/11/2020 asymptomatic', 'covid-19 (diagnosed 10/26/20)', 'covid-19  (diagnosed 10/26/20)'], 'covid-19')

In [ ]:
allall = []
for each in df['CUR_ILL']:
    if ',' in each:
        alls = each.split(',')
        ally = []
        for weach in alls:
            ally.append(weach.strip())
        allall.append(ally)
    else:
        allall.append(each)
        
allall2 = []
for each in allall:
    if type(each) == list:
        welt = "|".join(each)
        allall2.append(welt)
    else:
        allall2.append(each)
        
listy = list(pd.Series(allall2).value_counts()[pd.Series(allall2).value_counts()>13].index)
allall3 = list(map(lambda x: 'none' if x not in listy else x, allall2))
    
datufrayme = pd.Series(allall3).str.get_dummies()

listy.remove('alcohol use disorder|facial laceration|alcohol intoxication|secondary syphillis')
listy.remove('elevated troponin i level elevated troponin i level        elevated brain natriuretic peptide (bnp) level elevated brain natriuretic peptide (bnp) level        dyspnea       chest pain        atrial fibrillation with rapid ventricular response (hcc) atrial fibrillation with rapid ventricular response|initial encounter       hyponatremia hyponatremia')
    
df = df.join(datufrayme[listy], lsuffix=" cur_ill")

### Patient history

In [ ]:
df['HISTORY'] = df['HISTORY'].str.lower()

In [ ]:
# Quite a few entries for history need to be rectified, doubles were found using .value_counts() as we use the most common
nonelist = ['no', 'unknown', 'none.', 'none reported', 'n/a', 'na', 'none known', 'denies', 'none noted', '0', 'no illness',
           'none listed', 'not known', 'no known', 'non', 'no acute illnesses', 'no.', 'denied', 'see below', 'no illnesses',
            'unk', 'unkown', 'none documented', 'none stated', 'nothing', 'none known.', 'unknown.', 'as above', 'no known illnesses',
            'n/a.','no e', 'none reported.', 'medical history/concurrent conditions: no adverse event (no reported medical history)',
           'medical history/concurrent conditions: no adverse event (no reported medical history.)', 'see above', 'medical history/concurrent conditions: no adverse event',
           'medical history/concurrent conditions: no adverse event (no medical history reported.)', 'medical history/concurrent conditions: no adverse event (no medical history reported)',
           'medical history/concurrent conditions: no adverse event (medical history not provided)', 'comments: list of non-encoded patient relevant history: patient other relevant history 1: none',
           ]

df['HISTORY'] = df['HISTORY'].fillna('none')
df['HISTORY'] = df['HISTORY'].replace(nonelist, 'none')
df['HISTORY'] = df['HISTORY'].replace('medical history/concurrent conditions: covid-19', 'covid-19')
df['HISTORY'] = df['HISTORY'].replace('medical history/concurrent conditions: hypertension', 'hypertension')
df['HISTORY'] = df['HISTORY'].replace('medical history/concurrent conditions: penicillin allergy', 'penicillin allergy')
df['HISTORY'] = df['HISTORY'].replace(['medical history/concurrent conditions: asthma','mild asthma','exercise induced asthma'], 'asthma')
df['HISTORY'] = df['HISTORY'].replace('medical history/concurrent conditions: blood pressure high', 'high blood pressure')
df['HISTORY'] = df['HISTORY'].replace('medical history/concurrent conditions: sulfonamide allergy', 'sulfonamide allergy')
df['HISTORY'] = df['HISTORY'].replace(['diabetic', 'type 2 diabetes', 'type 1 diabetes'], 'diabetes')
df['HISTORY'] = df['HISTORY'].replace('medical history/concurrent conditions: migraine', 'migraines')

In [ ]:
allall = []
for each in df['HISTORY']:
    if ',' in each:
        alls = each.split(',')
        ally = []
        for weach in alls:
            ally.append(weach.strip())
        allall.append(ally)
    else:
        allall.append(each)
            
allall2 = []
for each in allall:
    if type(each) == list:
        welt = "|".join(each)
        allall2.append(welt)
    else:
        allall2.append(each)
            
listy = list(pd.Series(allall2).value_counts()[pd.Series(allall2).value_counts()>40].index)
allall3 = list(map(lambda x: 'none' if x not in listy else x, allall2))
    
datufrayme = pd.Series(allall3).str.get_dummies()

listy.remove('cerebral palsy|anxiety|crohns|bipolar|gerd|nutrition deficiency|iron deficiency')
    
df = df.join(datufrayme[listy], lsuffix=" history")

### Medication

In [ ]:
df['OTHER_MEDS'] = df['OTHER_MEDS'].str.lower()

In [ ]:
nonelist = ['unknown', 'no', 'none.', 'n/a', 'none reported', 'unk', 'none known', ';', 'not known', 'na', 'denies', ';  ;', 
           'nothing']

df['OTHER_MEDS'] = df['OTHER_MEDS'].fillna('none')
df['OTHER_MEDS'] = df['OTHER_MEDS'].replace(nonelist, 'none')

In [ ]:
allall = []
for each in df['OTHER_MEDS']:
    if ',' in each:
        alls = each.split(',')
        ally = []
        for weach in alls:
            ally.append(weach.strip())
        allall.append(ally)
    else:
        allall.append(each)
            
allall2 = []
for each in allall:
    if type(each) == list:
        welt = "|".join(each)
        allall2.append(welt)
    else:
        allall2.append(each)
            
listy = list(pd.Series(allall2).value_counts()[pd.Series(allall2).value_counts()>20].index)
allall3 = list(map(lambda x: 'none' if x not in listy else x, allall2))
    
datufrayme = pd.Series(allall3).str.get_dummies()
    
df = df.join(datufrayme[listy], lsuffix=" meds")

### Further Cleaning

In [ ]:
# drop all the columns with data that has been dummied, data that gives away the target such as days in hopital, 
#     columns of post-diagnosis information, and irrelavant colums such as Form Version
df.drop(columns = ['RECVDATE', 'CAGE_MO', 'CAGE_YR', 'RPT_DATE', 'SYMPTOM_TEXT', 'DIED',
                  'DATEDIED', 'L_THREAT', 'ER_VISIT', 'HOSPDAYS', 'X_STAY', 'RECOVD', 'VAX_DATE', 'ONSET_DATE',
                  'NUMDAYS', 'LAB_DATA', 'V_FUNDBY', 'OTHER_MEDS', 'CUR_ILL', 'HISTORY', 'PRIOR_VAX', 'SPLTTYPE', 
                  'FORM_VERS', 'TODAYS_DATE', 'OFC_VISIT', 'ER_ED_VISIT', 'ALLERGIES', 'VAX_TYPE', 'VAX_LOT', 
                  'SYMPTOM1', 'SYMPTOM2','SYMPTOM3', 'SYMPTOM4', 'SYMPTOM5','SYMPTOMVERSION1', 'SYMPTOMVERSION2',
                   'SYMPTOMVERSION3','SYMPTOMVERSION4', 'SYMPTOMVERSION5', 'VAX_NAME', 'VAERS_ID'],
       axis = 1, inplace=True)

In [ ]:
df = df.astype(int)

In [ ]:
# categoricals = ['STATE', 'V_ADMINBY', 'VAX_MANU', 'VAX_ROUTE', 'VAX_SITE']
# df[categoricals] = df[categoricals].astype('category')

In [ ]:
# df = df.drop_duplicates()

# EDA

# Modelling

In [ ]:
# these columns do not need to be dummied, passing enable_categorical=True into the DMatrixs will allow it to process, but only
#     if gpu is enabled and numerous other requirements are met.
df = pd.concat([df,pd.get_dummies(df['STATE'], prefix='STATE: ')],axis=1).drop(['STATE'],axis=1)
df = pd.concat([df,pd.get_dummies(df['VAX_MANU'], prefix='BRAND: ')],axis=1).drop(['VAX_MANU'],axis=1)
df = pd.concat([df,pd.get_dummies(df['VAX_SITE'], prefix='VAX_SITE: ')],axis=1).drop(['VAX_SITE'],axis=1)
df = pd.concat([df,pd.get_dummies(df['VAX_ROUTE'], prefix='VAX_ROUTE: ')],axis=1).drop(['VAX_ROUTE'],axis=1)
df = pd.concat([df,pd.get_dummies(df['V_ADMINBY'], prefix='ADMINBY: ')],axis=1).drop(['V_ADMINBY'],axis=1)

In [ ]:
# lets use hospitalization as our target, if it performs well enough we can try the continious target hosp_days
x = df.drop(columns = ['HOSPITAL'])
y = df['HOSPITAL']
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = .2)
xtrain, xval, ytrain, yval = train_test_split(xtrain, ytrain, test_size = .2)

### SMOTE

In [ ]:
# Continuing with Smote greatly increases our metrics on the train data, but returns the worst results we have seen on the val
from imblearn.over_sampling import SMOTE
sm = SMOTE()
X_train_oversampled, ytrain = sm.fit_sample(xtrain, ytrain)
xtrain = pd.DataFrame(X_train_oversampled, columns=xtrain.columns)

In [ ]:
# adasyn is an alternative to smote, aulthough they are very similar it is still worth attempting
# from imblearn.over_sampling import ADASYN
# ada = ADASYN()
# X_train_oversampled, ytrain = ada.fit_sample(xtrain, ytrain)
# xtrain = pd.DataFrame(X_train_oversampled, columns=xtrain.columns)

In [ ]:
from sklearn.linear_model import LogisticRegression 

In [ ]:
lr = LogisticRegression()
lr.fit(xtrain, ytrain)

In [ ]:
lr.score(xtrain, ytrain)

In [ ]:
preds = lr.predict(xtrain)

In [ ]:
recall_score(ytrain, preds)

In [ ]:
yhat_val = lr.predict(xval)
recall_score(yval, yhat_val)

In [ ]:
plot_confusion_matrix(lr, xval, yval)

In [ ]:
lr_las = LogisticRegression(penalty='l1', C=.001, solver='liblinear')
lr_las.fit(xtrain, ytrain)

In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgb = XGBClassifier(n_estimators = 300, max_depth=5)
xgb.fit(xtrain, ytrain)
yhat_train = xgb.predict(xtrain)
recall_score(ytrain, yhat_train)

In [ ]:
yhat_val = xgb.predict(xval)
recall_score(yval, yhat_val)

In [ ]:
# lets try using the simpler RandomOverSampler instead
# from imblearn.over_sampling import RandomOverSampler
# ros = RandomOverSampler()
# X_train_oversampled, ytrain = ros.fit_sample(xtrain, ytrain)
# xtrain = pd.DataFrame(X_train_oversampled, columns=xtrain.columns)

In [ ]:
# hm, none of our oversampling technique improves our mode, maybe undersampling the larger of the classes?
# from imblearn.under_sampling import RandomUnderSampler
# rus = RandomUnderSampler()
# X_train_oversampled, ytrain = rus.fit_sample(xtrain, ytrain)
# xtrain = pd.DataFrame(X_train_oversampled, columns=xtrain.columns)

In [ ]:
# Unfortunatley it seems that sampling our train data makes our model perform very poorly on the validation set

### Lazy Predict

In [ ]:
# Lazy Predict is a great tool to easily test your dataset across over thirty models
# this gives us a head start in our modelling as we can begin from the best result below
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(xtrain, xval, ytrain, yval)

print(models)

### SKLearn Random Forest

In [ ]:
# since Random Forests generated the best results, we should try and improve upon it
rfc = RandomForestClassifier('n_estimators': 100,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap)
rfc.fit(xtrain, ytrain)

In [ ]:
ypred = rfc.predict(xtrain)
plot_confusion_matrix(estimator=rfc, y_true=ytrain, X = xtrain)

In [ ]:
ypred = rfc.predict(xval)
plot_confusion_matrix(estimator=rfc, y_true=yval, X = xval)

In [ ]:
recall_score(yval, ypred)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
rfc = RandomForestClassifier()

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

In [ ]:
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
from pprint import pprint

In [ ]:
pprint(random_grid)

In [ ]:
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 8, cv = 3, verbose=3, n_jobs = -1)
rf_random.fit(xtrain, ytrain)

In [ ]:
rf_random.best_params_

In [ ]:
def evaluate(model, test_features, test_labels):
#     predictions = model.predict(test_features)
    return plot_confusion_matrix(model,test_features,test_labels, values_format = 'd')
    
#     errors = abs(predictions - test_labels)
#     mape = 100 * np.mean(errors / test_labels)
#     accuracy = 100 - mape
#     print('Model Performance')
#     print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
#     print('Accuracy = {:0.2f}%.'.format(accuracy))
    
#     return accuracy

In [ ]:
base_model = RandomForestClassifier(n_estimators = 10, random_state = 42)
base_model.fit(xtrain, ytrain)
base_accuracy = evaluate(base_model, xval, yval)

In [ ]:
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, xval, yval)

In [ ]:
plot_confusion_matrix([[8707,  365],[ 937, 1264]])

## TREES BAYES

In [ ]:
import sklearn
sklearn.__version__

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
def rf_cv(X, y, **kwargs):
    estimator = RandomForestClassifier(**kwargs)
    cval = cross_val_score(estimator, X, y, scoring = 'roc_auc', cv = 4, verbose = 3, n_jobs = -1)
    return cval.mean()

In [ ]:
def bayesian_optimise_rf(X, y, n_iter = 100):
    def rf_crossval(n_estimators, max_features, max_depth, min_samples_split, min_samples_leaf):
        #Wrapper of RandomForest cross validation.
        #Note the fixing of the inputs so they match the expected type
        #(e.g n_estimators must be an integer)
        return rf_cv(
            X = X,
            y = y,
            class_weight = "balanced",
            criterion = "entropy",
            min_samples_leaf = round(min_samples_leaf),
            min_samples_split = round(min_samples_split),
            max_depth = round(max_depth),
            n_estimators = round(n_estimators),
            max_features = max(min(max_features, 0.999), 1e-3),
            bootstrap = True
        )
    
    optimizer = BayesianOptimization(
        f = rf_crossval,
        pbounds = {
            'max_depth': (5, 40),
            'min_samples_split': (1, 40),
            'min_samples_leaf': (1, 40),
            "n_estimators" : (10, 1000),
            "max_features" : (0.1, 0.999),
        }
    )
    optimizer.maximize(n_iter = n_iter, init_points=10)
    print("Final result:", optimizer.max)

In [ ]:
bayesian_optimise_rf(xtrain, ytrain, n_iter = 100)

In [ ]:
Final result: {'target': -0.07562485649557907, 'params': {'max_features': 0.14536396029573667, 'n_estimators': 274.5153933464836}}
Final result: {'target': -0.10702808396076546, 'params': {'max_depth': 16.510516494808947, 'max_features': 0.9747272567894322, 'min_samples_leaf': 3.655444204049038, 'min_samples_split': 2.290201118341872, 'n_estimators': 212.19037867335334}}
Final result: {'target': -0.09605024365610514, 'params': {'max_depth': 19.140093166281204, 'max_features': 0.4629993256585475, 'min_samples_leaf': 1.2047202438817228, 'min_samples_split': 3.034554465488434, 'n_estimators': 168.6328805540666}}
    |  8        | -0.1029   |  27.54    |  0.999    |  1.0      |  22.93    |  687.6    |
    |  10       | -0.08651  |  28.96    |  0.999    |  1.0      |  4.866    |  394.7    |
    |  13       |  0.7182   |  40.0     |  0.1      |  7.443    |  40.0     |  290.6    |
    |  16       |  0.8935   |  34.12    |  0.8473   |  1.495    |  1.954    |  135.4    |


In [ ]:
base_model = RandomForestClassifier(n_estimators = 135,
                                   max_depth= 34.12, max_features= 0.8473,
                                    min_samples_leaf= 1, min_samples_split= 2,
                                   random_state=888)
base_model.fit(xtrain, ytrain)
base_accuracy = evaluate(base_model, xval, yval)

In [ ]:
base_model = RandomForestClassifier(n_estimators = 135, random_state=888)
base_model.fit(xtrain, ytrain)
base_accuracy = evaluate(base_model, xval, yval)

In [ ]:
base_model = RandomForestClassifier(random_state=888, max_features= 0.5)
base_model.fit(xtrain, ytrain)
base_accuracy = evaluate(base_model, xval, yval)

## TREES BAYES

In [ ]:
feauture_imp= list(zip(rfc.feature_importances_, xval.columns))
sorted(feauture_imp)

### XGBoost

In [ ]:
# since the best performing model was a random forest, we can try XGBoost as it is also based on random forests

In [ ]:
import xgboost as xgb

In [ ]:
df = df.astype(int)

In [ ]:
# XGBoost requires it own matrix type to be used
# XGBoost was built with sparse data in mind, adding the missing paramater will greatly improve the efficiency of the training
dtrain = xgb.DMatrix(xtrain, label=ytrain, missing=0)
dtest  = xgb.DMatrix(xtest, label=ytest, missing=0)
dval   = xgb.DMatrix(xval, label=yval, missing=0)

In [ ]:
# lets define a parameter tuner using Bayesian Optimiization
def bo_tune_xgb(max_depth, gamma ,learning_rate, scale_pos_weight, min_child_weight, colsample_bytree, subsample, alpha, lambd):
    params = {'max_depth'       : int(max_depth),
              'gamma'           : gamma,
#               'n_estimators'    : int(n_estimators),
              'learning_rate'   : learning_rate,
              'alpha'           : alpha,
              'lambda'          : lambd,
              'subsample'       : subsample,
              'eval_metric'     : 'rmse',
              'min_child_weight': min_child_weight,
              'scale_pos_weight': scale_pos_weight,
              'colsample_bytree': colsample_bytree,
              'tree_method'     : 'gpu_hist'}
    cv_result = xgb.cv(params, dtrain, num_boost_round=200, nfold=5)
    return -1.0 * cv_result['test-rmse-mean'].iloc[-1]

In [ ]:
# here we define the ranges that Bayesian Optimization is allowed to search through
# these ranges are common parameter ranges that most models fall into
xgb_bo = BayesianOptimization(bo_tune_xgb, {'max_depth' : (1, 20),
                        'gamma'            : (0, 2),
                        'alpha'            : (0, 10),
                        'lambd'           : (0,3),
                        'subsample'        : (0,1),           
                        'learning_rate'    : (0,2),
#                         'n_estimators'     : (100,400),
                        'scale_pos_weight' : (6,7),
                        'min_child_weight' : (1,10),
                        'colsample_bytree' : (0,1)} ,verbose=3)

In [ ]:
# here we search for the strongest parameter combination
xgb_bo.maximize(n_iter=7, init_points=5, acq='ei')

In [ ]:
# .max returns the strongest parameters
params = xgb_bo.max['params']

# some of the parameters need to be whole numbers
params['max_depth'] = int(params['max_depth'])
# params['n_estimators'] = int(params['n_estimators'])
params

In [ ]:
# lets see our results for the train data
xgb_opt= xgb.train(params, dtrain)
predsopt = xgb_opt.predict(dtrain)
print(classification_report(predsopt.round(), ytrain))

In [ ]:
confusion_matrix(predsopt.round(), ytrain)

In [ ]:
# and now for the validation
predsoptval = xgb_opt.predict(dval)
print(classification_report(predsoptval.round(), yval))

In [ ]:
confusion_matrix(predsoptval.round(), yval)

In [ ]:
predsoptval = xgb_opt.predict(dval)
print(classification_report(predsoptval.round(), yval))

### Neural Network

In [ ]:
ypred = xgb_opt.predict(dval)
plot_confusion_matrix([[8707,  365],[ 937, 1264]])

In [ ]:
# swish is a recent activation function that remedies the issues of ReLU
def swish(x, b = 1):
    return (x * sigmoid(b * x))

In [ ]:
def newmod():
    model = tf.keras.Sequential()
    model.add(Dense(176, input_dim=len(xtrain.columns), activation='swish'))
    model.add(Dropout(.2))

    model.add(Dense(88, activation='swish'))
    model.add(Dropout(.2))
    
    model.add(Dense(44, activation='swish'))
    model.add(Dropout(.2))
    
    model.add(Dense(22, activation='swish'))
    model.add(Dropout(.2))
    
    model.add(Dense(11, activation='swish'))
    model.add(Dropout(.2))
    
    model.add(Dense(1, activation='sigmoid'))
    
    return model


estimator = newmod()
estimator.compile(optimizer='nadam', 
                  metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()], loss='binary_crossentropy')

In [ ]:
history = estimator.fit(xtrain, ytrain, epochs=100, validation_data=(xval, yval))

In [ ]:
history_df = pd.DataFrame(history.history)
plt.plot(history_df['loss'], label='loss')
plt.plot(history_df['val_loss'], label='val_loss')

plt.legend()

# catboost

In [ ]:
import catboost as cgb


In [ ]:
def cat_hyp(depth, bagging_temperature, iterations, learning_rate, random_strength, border_count):
  params = {'depth': depth,
#             'iterations' : int(iterations),
            'learning_rate' : learning_rate,
            'random_strength' : random_strength,
            'bagging_temperature': bagging_temperature,
            'border_count' : int(border_count),
#             '12_leaf_reg' : leaf_reg,
#             'scale_pos_weight' : scale_pos_weight,
            "eval_metric": "F1",
            "loss_function" : "Logloss",
            "verbose": True,
            "task_type" : "GPU",
            "devices" : '0:1'}
  params[ "depth"] = int(round(depth)) 
  params["bagging_temperature"] = bagging_temperature
#   params['iterations'] = int(iterations)
  params['learning_rate'] = learning_rate
  params['random_strength'] = random_strength
#   params['']
  
  cat_feat = [] # Categorical features list
  cv_dataset = cgb.Pool(data=xtrain,
                  label=ytrain,
                  cat_features=cat_feat)

  scores = cgb.cv(cv_dataset,
                  params,
                  plot=True,
                  fold_count=3)
  return np.max(scores['test-F1-mean'])

In [ ]:
pds = {'depth': (0, 10),
       'iterations' : (500, 600),
       'learning_rate' : (.01, 1),
       'random_strength' : (.0000001,10),
       'bagging_temperature': (0,1),
       'border_count' : (1,255)
#        '12_leaf_reg' : (2, 30),
#        'scale_pos_weight' : (.01, 1)
          }

In [ ]:
# Surrogate model
optimizer = BayesianOptimization(cat_hyp, pds, random_state=2100)
                                  
# Optimize
optimizer.maximize(init_points=3, n_iter=7)

In [ ]:
params = optimizer.max['params']

In [ ]:
params['depth'] = int(params['depth'])
params['iterations'] = int(params['iterations'])
params['border_count'] = int(params['border_count'])


params

In [ ]:
cat_feat = []
cv_dataset = cgb.Pool(data=xtrain,
                label=ytrain.values)

cgb_opt = cgb.train( cv_dataset, plot=True)
predsopt = cgb_opt.predict(cv_dataset)
print(classification_report(predsopt.round(), ytrain))

In [ ]:
confusion_matrix(predsopt.round(), ytrain)

In [ ]:
opt = BayesSearchCV(clf,
                    search_spaces,
                    scoring=roc_auc,
                    cv=skf,
                    n_iter=100,
                    n_jobs=1,  # use just 1 job with CatBoost in order to avoid segmentation fault
                    return_train_score=False,
                    refit=True,
                    optimizer_kwargs={'base_estimator': 'GP'},
                    random_state=42)

In [ ]:
# %%time
tuned_model = CatBoostClassifier(**best_params,task_type = "GPU",od_type='Iter',one_hot_max_size=10)
tuned_model.fit(X_train,y_train)

In [ ]:
y_pred = tuned_model.predict_proba(X_valid)[:, 1]
valid_score = roc_auc_score(y_valid, y_pred)
print('Validation ROC-AUC score:', valid_score)

In [ ]:
ytrain.sum()

In [ ]:
len(ytrain)

In [ ]:
tuned_model = CatBoostClassifier(**best_params,task_type = "GPU",od_type='Iter',one_hot_max_size=10)
tuned_model.fit(df_train_features,y_target)

In [4]:
import pandas as pd

In [5]:
colnames = ['AGE_YRS','SEX','HOSPITAL','DISABLE','BIRTH_DEFECT','VAX_DOSE_SERIES','none', 'cur_ill','nka','nkda','penicillin','sulfa','pcn','latex','codeine','amoxicillin','sulfa drugs','shellfish','morphine','bactrim','seasonal allergies cur_ill','erythromycin','aspirin','sulfa antibiotics','augmentin','lisinopril meds','lactose','denies','penicillins','bee stings','cipro','iodine','nickel','levaquin','seasonal','ceclor','percocet','compazine','gluten','nkma','doxycycline','nsaids','keflex','ibuprofen meds','naproxen','codiene','tree nuts','bee venom','peanuts','clindamycin','none history','covid-19 history','asthma history','uti','seasonal allergies history','sinus infection','hypertension history','diabetes history','migraines history','blood pressure high','urinary tract infection','migraine history','hypothyroidism history','allergies','anxiety history','diabetic','gerd history','cold','sinusitis','acid reflux','copd history','new diagnosis of t2dm','fibromyalgia history','rheumatoid arthritis history','none meds','asthma','hypertension','hypothyroidism','diabetes','high blood pressure','migraines','htn','covid-19','anxiety','gerd','obesity','depression','hypothyroid','arthritis','rheumatoid arthritis','high cholesterol','copd','seasonal allergies','penicillin allergy','fibromyalgia','migraine','pcos','hyperlipidemia','lupus','ibs','sulfonamide allergy','ulcerative colitis','psoriasis','adhd','none','multivitamin','tylenol','synthroid','levothyroxine','birth control','ibuprofen','vitamin d','lexapro','prenatal vitamins','zyrtec','adderall','zoloft','vitamins','lisinopril','prenatal vitamin','metformin','multi vitamin','multivitamins','losartan','prozac','wellbutrin','flonase','eliquis','metoprolol','vitamin c','nuvaring','atorvastatin','omeprazole','melatonin','insulin','birth control pill','multi-vitamin','mirena iud','acetaminophen','amlodipine','daily multivitamin','prilosec','sertraline','albuterol','vitamin d3','birth control pills','coumadin','allegra','benadryl','STATE: _0','STATE: _1','STATE: _2','STATE: _3','STATE: _4','STATE: _5','STATE: _6','STATE: _7','STATE: _8','STATE: _9','STATE: _10','STATE: _11','STATE: _12','STATE: _13','STATE: _14','STATE: _15','STATE: _16','STATE: _17','STATE: _18','STATE: _19','STATE: _20','STATE: _21','STATE: _22','STATE: _23','STATE: _24','STATE: _25','STATE: _26','STATE: _27','STATE: _28','STATE: _29','STATE: _30','STATE: _31','STATE: _32','STATE: _33','STATE: _34','STATE: _35','STATE: _36','STATE: _37','STATE: _38','STATE: _39','STATE: _40','STATE: _41','STATE: _42','STATE: _43','STATE: _44','STATE: _45','STATE: _46','STATE: _47','STATE: _48','STATE: _49','STATE: _50','STATE: _51','STATE: _52','STATE: _53','BRAND: _0','BRAND: _1','BRAND: _2','BRAND: _3','VAX_SITE: _0','VAX_SITE: _1','VAX_SITE: _2','VAX_SITE: _3','VAX_SITE: _4','VAX_SITE: _5','VAX_SITE: _6','VAX_SITE: _7','VAX_SITE: _8','VAX_SITE: _9','VAX_ROUTE: _0','VAX_ROUTE: _1','VAX_ROUTE: _2','VAX_ROUTE: _3','VAX_ROUTE: _4','VAX_ROUTE: _5','VAX_ROUTE: _6','ADMINBY: _0','ADMINBY: _1','ADMINBY: _2','ADMINBY: _3','ADMINBY: _4','ADMINBY: _5','ADMINBY: _6','ADMINBY: _7','ADMINBY: _8']
df = pd.DataFrame(columns=colnames)

In [30]:
for each in colnames:
    print(f'{each} :  <input type="checkbox", name={chr(39)}{each}{chr(39)}> <br>')

AGE_YRS :  <input type="checkbox", name='AGE_YRS'> <br>
SEX :  <input type="checkbox", name='SEX'> <br>
DISABLE :  <input type="checkbox", name='DISABLE'> <br>
BIRTH_DEFECT :  <input type="checkbox", name='BIRTH_DEFECT'> <br>
VAX_DOSE_SERIES :  <input type="checkbox", name='VAX_DOSE_SERIES'> <br>
none :  <input type="checkbox", name='none'> <br>
cur_ill :  <input type="checkbox", name='cur_ill'> <br>
nka :  <input type="checkbox", name='nka'> <br>
nkda :  <input type="checkbox", name='nkda'> <br>
penicillin :  <input type="checkbox", name='penicillin'> <br>
sulfa :  <input type="checkbox", name='sulfa'> <br>
pcn :  <input type="checkbox", name='pcn'> <br>
latex :  <input type="checkbox", name='latex'> <br>
codeine :  <input type="checkbox", name='codeine'> <br>
amoxicillin :  <input type="checkbox", name='amoxicillin'> <br>
sulfa drugs :  <input type="checkbox", name='sulfa drugs'> <br>
shellfish :  <input type="checkbox", name='shellfish'> <br>
morphine :  <input type="checkbox", name

In [52]:
for each in colnames:
    print("try:")
    print(f'    df[{chr(39)}{each}{chr(39)}] = request.form[{chr(39)}{each}{chr(39)}]')
    print('except:')
    print("    pass")

try:
    df['AGE_YRS'] = request.form['AGE_YRS']
except:
    pass
try:
    df['SEX'] = request.form['SEX']
except:
    pass
try:
    df['DISABLE'] = request.form['DISABLE']
except:
    pass
try:
    df['BIRTH_DEFECT'] = request.form['BIRTH_DEFECT']
except:
    pass
try:
    df['VAX_DOSE_SERIES'] = request.form['VAX_DOSE_SERIES']
except:
    pass
try:
    df['none'] = request.form['none']
except:
    pass
try:
    df['cur_ill'] = request.form['cur_ill']
except:
    pass
try:
    df['nka'] = request.form['nka']
except:
    pass
try:
    df['nkda'] = request.form['nkda']
except:
    pass
try:
    df['penicillin'] = request.form['penicillin']
except:
    pass
try:
    df['sulfa'] = request.form['sulfa']
except:
    pass
try:
    df['pcn'] = request.form['pcn']
except:
    pass
try:
    df['latex'] = request.form['latex']
except:
    pass
try:
    df['codeine'] = request.form['codeine']
except:
    pass
try:
    df['amoxicillin'] = request.form['amoxicillin']
except:
    pass
try:
    df['s

In [25]:
colnames = ['AGE_YRS','SEX','DISABLE','BIRTH_DEFECT','VAX_DOSE_SERIES','none', 'cur_ill','nka','nkda','penicillin','sulfa','pcn','latex','codeine','amoxicillin','sulfa drugs','shellfish','morphine','bactrim','seasonal allergies cur_ill','erythromycin','aspirin','sulfa antibiotics','augmentin','lisinopril meds','lactose','denies','penicillins','bee stings','cipro','iodine','nickel','levaquin','seasonal','ceclor','percocet','compazine','gluten','nkma','doxycycline','nsaids','keflex','ibuprofen meds','naproxen','codiene','tree nuts','bee venom','peanuts','clindamycin','none history','covid-19 history','asthma history','uti','seasonal allergies history','sinus infection','hypertension history','diabetes history','migraines history','blood pressure high','urinary tract infection','migraine history','hypothyroidism history','allergies','anxiety history','diabetic','gerd history','cold','sinusitis','acid reflux','copd history','new diagnosis of t2dm','fibromyalgia history','rheumatoid arthritis history','none meds','asthma','hypertension','hypothyroidism','diabetes','high blood pressure','migraines','htn','covid-19','anxiety','gerd','obesity','depression','hypothyroid','arthritis','rheumatoid arthritis','high cholesterol','copd','seasonal allergies','penicillin allergy','fibromyalgia','migraine','pcos','hyperlipidemia','lupus','ibs','sulfonamide allergy','ulcerative colitis','psoriasis','adhd','none','multivitamin','tylenol','synthroid','levothyroxine','birth control','ibuprofen','vitamin d','lexapro','prenatal vitamins','zyrtec','adderall','zoloft','vitamins','lisinopril','prenatal vitamin','metformin','multi vitamin','multivitamins','losartan','prozac','wellbutrin','flonase','eliquis','metoprolol','vitamin c','nuvaring','atorvastatin','omeprazole','melatonin','insulin','birth control pill','multi-vitamin','mirena iud','acetaminophen','amlodipine','daily multivitamin','prilosec','sertraline','albuterol','vitamin d3','birth control pills','coumadin','allegra','benadryl','STATE: _0','STATE: _1','STATE: _2','STATE: _3','STATE: _4','STATE: _5','STATE: _6','STATE: _7','STATE: _8','STATE: _9','STATE: _10','STATE: _11','STATE: _12','STATE: _13','STATE: _14','STATE: _15','STATE: _16','STATE: _17','STATE: _18','STATE: _19','STATE: _20','STATE: _21','STATE: _22','STATE: _23','STATE: _24','STATE: _25','STATE: _26','STATE: _27','STATE: _28','STATE: _29','STATE: _30','STATE: _31','STATE: _32','STATE: _33','STATE: _34','STATE: _35','STATE: _36','STATE: _37','STATE: _38','STATE: _39','STATE: _40','STATE: _41','STATE: _42','STATE: _43','STATE: _44','STATE: _45','STATE: _46','STATE: _47','STATE: _48','STATE: _49','STATE: _50','STATE: _51','STATE: _52','STATE: _53','BRAND: _0','BRAND: _1','BRAND: _2','BRAND: _3','VAX_SITE: _0','VAX_SITE: _1','VAX_SITE: _2','VAX_SITE: _3','VAX_SITE: _4','VAX_SITE: _5','VAX_SITE: _6','VAX_SITE: _7','VAX_SITE: _8','VAX_SITE: _9','VAX_ROUTE: _0','VAX_ROUTE: _1','VAX_ROUTE: _2','VAX_ROUTE: _3','VAX_ROUTE: _4','VAX_ROUTE: _5','VAX_ROUTE: _6','ADMINBY: _0','ADMINBY: _1','ADMINBY: _2','ADMINBY: _3','ADMINBY: _4','ADMINBY: _5','ADMINBY: _6','ADMINBY: _7','ADMINBY: _8']
dfx = pd.DataFrame(columns=colnames) 
dfx['SEX'] = 1

In [27]:
dfx = dfx.append(pd.Series(0, index=dfx.columns), ignore_index=True)

In [31]:
df

,AGE_YRS,SEX,HOSPITAL,DISABLE,BIRTH_DEFECT,VAX_DOSE_SERIES,none,cur_ill,nka,nkda,...,VAX_ROUTE: _6,ADMINBY: _0,ADMINBY: _1,ADMINBY: _2,ADMINBY: _3,ADMINBY: _4,ADMINBY: _5,ADMINBY: _6,ADMINBY: _7,ADMINBY: _8


In [61]:
colnames = ['AGE_YRS','SEX','DISABLE','BIRTH_DEFECT','VAX_DOSE_SERIES','none', 'cur_ill','nka','nkda','penicillin','sulfa','pcn','latex','codeine','amoxicillin','sulfa drugs','shellfish','morphine','bactrim','seasonal allergies cur_ill','erythromycin','aspirin','sulfa antibiotics','augmentin','lisinopril meds','lactose','denies','penicillins','bee stings','cipro','iodine','nickel','levaquin','seasonal','ceclor','percocet','compazine','gluten','nkma','doxycycline','nsaids','keflex','ibuprofen meds','naproxen','codiene','tree nuts','bee venom','peanuts','clindamycin','none history','covid-19 history','asthma history','uti','seasonal allergies history','sinus infection','hypertension history','diabetes history','migraines history','blood pressure high','urinary tract infection','migraine history','hypothyroidism history','allergies','anxiety history','diabetic','gerd history','cold','sinusitis','acid reflux','copd history','new diagnosis of t2dm','fibromyalgia history','rheumatoid arthritis history','none meds','asthma','hypertension','hypothyroidism','diabetes','high blood pressure','migraines','htn','covid-19','anxiety','gerd','obesity','depression','hypothyroid','arthritis','rheumatoid arthritis','high cholesterol','copd','seasonal allergies','penicillin allergy','fibromyalgia','migraine','pcos','hyperlipidemia','lupus','ibs','sulfonamide allergy','ulcerative colitis','psoriasis','adhd','none','multivitamin','tylenol','synthroid','levothyroxine','birth control','ibuprofen','vitamin d','lexapro','prenatal vitamins','zyrtec','adderall','zoloft','vitamins','lisinopril','prenatal vitamin','metformin','multi vitamin','multivitamins','losartan','prozac','wellbutrin','flonase','eliquis','metoprolol','vitamin c','nuvaring','atorvastatin','omeprazole','melatonin','insulin','birth control pill','multi-vitamin','mirena iud','acetaminophen','amlodipine','daily multivitamin','prilosec','sertraline','albuterol','vitamin d3','birth control pills','coumadin','allegra','benadryl','STATE: _0','STATE: _1','STATE: _2','STATE: _3','STATE: _4','STATE: _5','STATE: _6','STATE: _7','STATE: _8','STATE: _9','STATE: _10','STATE: _11','STATE: _12','STATE: _13','STATE: _14','STATE: _15','STATE: _16','STATE: _17','STATE: _18','STATE: _19','STATE: _20','STATE: _21','STATE: _22','STATE: _23','STATE: _24','STATE: _25','STATE: _26','STATE: _27','STATE: _28','STATE: _29','STATE: _30','STATE: _31','STATE: _32','STATE: _33','STATE: _34','STATE: _35','STATE: _36','STATE: _37','STATE: _38','STATE: _39','STATE: _40','STATE: _41','STATE: _42','STATE: _43','STATE: _44','STATE: _45','STATE: _46','STATE: _47','STATE: _48','STATE: _49','STATE: _50','STATE: _51','STATE: _52','STATE: _53','BRAND: _0','BRAND: _1','BRAND: _2','BRAND: _3','VAX_SITE: _0','VAX_SITE: _1','VAX_SITE: _2','VAX_SITE: _3','VAX_SITE: _4','VAX_SITE: _5','VAX_SITE: _6','VAX_SITE: _7','VAX_SITE: _8','VAX_SITE: _9','VAX_ROUTE: _0','VAX_ROUTE: _1','VAX_ROUTE: _2','VAX_ROUTE: _3','VAX_ROUTE: _4','VAX_ROUTE: _5','VAX_ROUTE: _6','ADMINBY: _0','ADMINBY: _1','ADMINBY: _2','ADMINBY: _3','ADMINBY: _4','ADMINBY: _5','ADMINBY: _6','ADMINBY: _7','ADMINBY: _8']
dfy = pd.DataFrame(columns=colnames)    
dfy = dfy.append(pd.Series(0, index=dfy.columns), ignore_index=True)

list(dfy.values)

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=object)]

In [62]:
22*10

220